# Try numba

Arrow doesn't seem to work in Numba. Float Numpy does, but by default I get an object array out which is weird!

In [1]:
import numba

In [2]:
import pandas as pd
import polars as pl
from humanfriendly import format_size, format_number
from simpler_mpl import set_commas, set_common_mpl_styles
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload 2

display(f"Pandas {pd.__version__}, Polars {pl.__version__}")

from cell_profiler import cell_profiler as cp

%start_cell_profiler


'Pandas 2.0.1, Polars 0.17.10'

'Profiling enabled'

In [2] used 0.0 MiB RAM in 0.11s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 264.0 MiB


In [3]:
dfpda = pd.read_parquet("../test_result_2021on.parquet", 
                        dtype_backend="pyarrow")

In [3] used 12864.0 MiB RAM in 14.34s (system mean cpu 44%, single max cpu 100%), peaked 3485.8 MiB above final usage, current RAM usage now 13127.9 MiB


In [40]:
dfpda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82012245 entries, 0 to 82012244
Data columns (total 14 columns):
 #   Column             Dtype                 
---  ------             -----                 
 0   test_id            int64[pyarrow]        
 1   vehicle_id         int64[pyarrow]        
 2   test_date          timestamp[ns][pyarrow]
 3   test_class_id      int64[pyarrow]        
 4   test_type          large_string[pyarrow] 
 5   test_result        large_string[pyarrow] 
 6   test_mileage       int64[pyarrow]        
 7   postcode_area      large_string[pyarrow] 
 8   make               large_string[pyarrow] 
 9   model              large_string[pyarrow] 
 10  colour             large_string[pyarrow] 
 11  fuel_type          large_string[pyarrow] 
 12  cylinder_capacity  int64[pyarrow]        
 13  first_use_date     timestamp[ns][pyarrow]
dtypes: int64[pyarrow](5), large_string[pyarrow](7), timestamp[ns][pyarrow](2)
memory usage: 10.4 GB
In [40] used 0.0 MiB RAM in 1.60

In [4]:
dfpdn = pd.read_parquet("../test_result_2021on.parquet", 
                        dtype_backend="numpy_nullable")

In [4] used 23951.7 MiB RAM in 35.89s (system mean cpu 39%, single max cpu 100%), peaked 600.5 MiB above final usage, current RAM usage now 37079.6 MiB


In [5]:
ser_mileage_pdn = dfpdn['test_mileage'].fillna(0).to_numpy()
type(ser_mileage_pdn)
#  3137.1 MiB RAM in 1.67s 

numpy.ndarray

In [5] used -8551.4 MiB RAM in 4.91s (system mean cpu 18%, single max cpu 100%), peaked 8053.1 MiB above final usage, current RAM usage now 28528.2 MiB


In [6]:
ser_mileage_pdn.dtype # !!! why object?!

dtype('O')

In [6] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 28528.2 MiB


In [7]:
ser_mileage_pdn = ser_mileage_pdn.astype('float')
ser_mileage_pdn.dtype

dtype('float64')

In [7] used -2480.5 MiB RAM in 2.36s (system mean cpu 9%, single max cpu 68%), peaked 0.0 MiB above final usage, current RAM usage now 26047.7 MiB


In [8]:
#dfpda['test_mileage'].to_numpy() # feather
# used 3157.7 MiB RAM in 2.68s

In [8] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26047.7 MiB


In [9]:
ser_mileage_pda = dfpda['test_mileage'].fillna(0).values
type(ser_mileage_pda)

pandas.core.arrays.arrow.array.ArrowExtensionArray

In [9] used 635.5 MiB RAM in 0.43s (system mean cpu 11%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26683.2 MiB


In [10]:
def do_thing(ser):
    # given ser, loop and find max
    mx = 0
    for idx in range(ser.shape[0]):
        value = ser[idx]
        if value > mx:
            mx = value
    return mx

In [10] used 0.0 MiB RAM in 0.10s (system mean cpu 8%, single max cpu 30%), peaked 0.0 MiB above final usage, current RAM usage now 26683.2 MiB


In [11]:
do_thing(ser_mileage_pdn)

999999.0

In [11] used 0.0 MiB RAM in 5.95s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26683.2 MiB


In [12]:
do_thing(ser_mileage_pda[:100000]) # crazy slow if not limited!

997046

In [12] used 0.0 MiB RAM in 0.28s (system mean cpu 8%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26683.2 MiB


In [13]:
from numba import njit
do_thing_njit = njit(do_thing)

In [13] used 3.4 MiB RAM in 0.11s (system mean cpu 8%, single max cpu 50%), peaked 0.0 MiB above final usage, current RAM usage now 26686.6 MiB


In [14]:
do_thing_njit(ser_mileage_pdn)

999999.0

In [14] used 22.8 MiB RAM in 0.44s (system mean cpu 5%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26709.5 MiB


In [15]:
#- argument 0: Cannot determine Numba type of <class 'pandas.core.arrays.arrow.array.ArrowExtensionArray'>
#do_thing_njit(ser_mileage_pda)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at /tmp/ipykernel_713535/654114077.py (1)[0m
[1m
File "../../../../../../tmp/ipykernel_713535/654114077.py", line 1:[0m
[1m<source missing, REPL/exec in use?>[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'pandas.core.arrays.arrow.array.ArrowExtensionArray'>[0m


In [15] used 5.1 MiB RAM in 0.37s (system mean cpu 8%, single max cpu 95%), peaked 0.0 MiB above final usage, current RAM usage now 26714.6 MiB


# Bit unsure about all of this!

In [19]:
import numpy as np

In [19] used 0.0 MiB RAM in 0.10s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26726.2 MiB


In [17]:
%timeit ser_mileage_pdn.mean()

50.6 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [17] used 0.0 MiB RAM in 4.22s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26725.9 MiB


In [20]:
%timeit np.nanmean(ser_mileage_pdn)

248 ms ± 25.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [20] used 0.3 MiB RAM in 2.11s (system mean cpu 9%, single max cpu 100%), peaked 625.6 MiB above final usage, current RAM usage now 26726.5 MiB


In [26]:
%timeit dfpdn['test_mileage'].fillna(0).mean()

329 ms ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [26] used 0.0 MiB RAM in 2.74s (system mean cpu 10%, single max cpu 100%), peaked 625.6 MiB above final usage, current RAM usage now 26731.6 MiB


In [29]:
dfpdn['test_mileage'].fillna(0).mean()

73559.0500931172

In [29] used 0.0 MiB RAM in 0.46s (system mean cpu 10%, single max cpu 100%), peaked 625.6 MiB above final usage, current RAM usage now 26731.6 MiB


In [28]:
dfpdn['test_mileage'].mean()

74115.13152002622

In [28] used 0.0 MiB RAM in 0.29s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26731.6 MiB


In [27]:
%timeit dfpdn['test_mileage'].mean()

178 ms ± 9.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
In [27] used 0.0 MiB RAM in 14.40s (system mean cpu 10%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 26731.6 MiB


In [24]:
%timeit dfpda['test_mileage'].fillna(0).mean()

338 ms ± 21.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
In [24] used 0.0 MiB RAM in 2.83s (system mean cpu 9%, single max cpu 100%), peaked 635.2 MiB above final usage, current RAM usage now 26731.9 MiB


# Trying sane item

TODO try to balance up speed timings on mean?

In [36]:
ser_pdn = pd.Series(np.array([0.1] * 1_000_000))
assert ser_pdn.dtype == np.dtype('float64')

In [36] used 0.0 MiB RAM in 0.16s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26732.6 MiB


In [44]:
ser_pda = ser_pdn.astype('float64[pyarrow]')
#assert ser_pda.dtype == np.dtype('double[pyarrow]')

In [44] used 0.1 MiB RAM in 0.10s (system mean cpu 4%, single max cpu 7%), peaked 0.0 MiB above final usage, current RAM usage now 26773.7 MiB


In [45]:
ser_pl = pl.from_pandas(ser)

In [45] used 0.0 MiB RAM in 0.10s (system mean cpu 4%, single max cpu 9%), peaked 0.0 MiB above final usage, current RAM usage now 26773.7 MiB


In [46]:
ser_pdn.mean()

0.0999999999999998

In [46] used -0.1 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 12%), peaked 0.0 MiB above final usage, current RAM usage now 26773.6 MiB


In [47]:
ser_pda.mean()

0.10000000000000003

In [47] used 0.0 MiB RAM in 0.10s (system mean cpu 5%, single max cpu 10%), peaked 0.0 MiB above final usage, current RAM usage now 26773.6 MiB


In [48]:
ser_pl.mean()

0.10000000000133288

In [48] used 0.0 MiB RAM in 0.14s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 26773.6 MiB
